In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [1]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("test")

LangSmith 추적을 시작합니다.
[프로젝트명]
test


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 모델을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template("{country} 에 대해서 200자 내외로 요약해줘")

# 체인을 생성합니다.
chain = prompt | llm


### Caching: 동일 질문에 대한 답변 저장 후, 활용 가능

In [4]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 인메모리 캐시를 사용합니다.
set_llm_cache(InMemoryCache())

# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 나라로, 서울이 수도이다. 인구는 약 5천만 명으로 밀집된 인구를 가지고 있으며, 경제적으로는 선진국으로 분류된다. 한국은 역사적으로 일본의 식민지였던 경험을 가졌고, 한반도 분단 이후 남한과 북한으로 나뉘어있다. 문화적으로는 한글과 한류 등이 유명하며, 전통적인 음식과 의상도 특징적이다. 한국은 기술과 연구 분야에서도 선진화되어 있으며, K-pop과 K-drama 등 한류 콘텐츠로 세계적으로 인기를 얻고 있다. 현대적인 도시와 전통적인 문화가 공존하는 매력적인 나라이다.
CPU times: user 276 ms, sys: 41.5 ms, total: 318 ms
Wall time: 2.72 s


In [5]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 나라로, 서울이 수도이다. 인구는 약 5천만 명으로 밀집된 인구를 가지고 있으며, 경제적으로는 선진국으로 분류된다. 한국은 역사적으로 일본의 식민지였던 경험을 가졌고, 한반도 분단 이후 남한과 북한으로 나뉘어있다. 문화적으로는 한글과 한류 등이 유명하며, 전통적인 음식과 의상도 특징적이다. 한국은 기술과 연구 분야에서도 선진화되어 있으며, K-pop과 K-drama 등 한류 콘텐츠로 세계적으로 인기를 얻고 있다. 현대적인 도시와 전통적인 문화가 공존하는 매력적인 나라이다.
CPU times: user 2.93 ms, sys: 282 µs, total: 3.21 ms
Wall time: 3.17 ms


### 직렬화: parsing 가능한 형태로 저장 및 load 하는 것

- dumps: 객체를 JSON 문자열로 직렬화

- dumpd: 객체를 딕셔너리로 직렬화

-> .pkl 파일로 저장 및 롣,

In [9]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# 프롬프트 템플릿을 사용하여 질문을 생성합니다.
prompt = PromptTemplate.from_template("{fruit}의 색상이 무엇입니까?")
# 직렬화가 가능한지 체크합니다.
print(f"ChatOpenAI: {ChatOpenAI.is_lc_serializable()}")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 직렬화가 가능한지 체크합니다.
print(f"ChatOpenAI: {llm.is_lc_serializable()}")

# 체인을 생성합니다.
chain = prompt | llm

# 직렬화가 가능한지 체크합니다.
chain.is_lc_serializable()

ChatOpenAI: True
ChatOpenAI: True


True

In [10]:
from langchain_core.load import dumpd, dumps

dumpd_chain = dumpd(chain)
dumpd_chain

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['fruit'],
    'template': '{fruit}의 색상이 무엇입니까?',
    'template_format': 'f-string'},
   'name': 'PromptTemplate'},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
   'kwargs': {'model_name': 'gpt-3.5-turbo',
    'temperature': 0.0,
    'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']},
    'max_retries': 2,
    'n': 1},
   'name': 'ChatOpenAI'}},
 'name': 'RunnableSequence'}

In [11]:
# dumps 함수를 사용하여 직렬화된 체인을 확인합니다.
dumps_chain = dumps(chain)
dumps_chain

'{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "runnable", "RunnableSequence"], "kwargs": {"first": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "prompt", "PromptTemplate"], "kwargs": {"input_variables": ["fruit"], "template": "{fruit}\\uc758 \\uc0c9\\uc0c1\\uc774 \\ubb34\\uc5c7\\uc785\\ub2c8\\uae4c?", "template_format": "f-string"}, "name": "PromptTemplate"}, "last": {"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "openai", "ChatOpenAI"], "kwargs": {"model_name": "gpt-3.5-turbo", "temperature": 0.0, "openai_api_key": {"lc": 1, "type": "secret", "id": ["OPENAI_API_KEY"]}, "max_retries": 2, "n": 1}, "name": "ChatOpenAI"}}, "name": "RunnableSequence"}'